In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Full MNIST Results 

## Auxiliary functions

In [2]:
def extract_metrics(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k)

def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line: 
            gpu_u, gpu_mem = eval(line.split(":")[-1])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [3]:
def calculate_metrics(files): 
    mlp_metrics = dict()
    cnn_metrics = dict()
    
    for file in files: 
        arch, method, i = file.split("_") 
        metrics = mlp_metrics if arch == "mlp" else cnn_metrics
        
        if method in metrics: 
            metrics[method].append(extract_metrics(file))
        else: 
            metrics[method] = [extract_metrics(file)] 
    
    return (mlp_metrics, cnn_metrics)

def calculate_metrics_gpu(files): 
    mlp_metrics = dict()
    cnn_metrics = dict()
    
    for file in files: 
        arch, method, i = file.split("_") 
        metrics = mlp_metrics if arch == "mlp" else cnn_metrics
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return (mlp_metrics, cnn_metrics)

In [4]:
def get_means(metrics): 
    return {method: [sum(x)/len(x) for x in list(zip(*metrics[method]))] for method in metrics}

# 1. Task-IL + GPU

In [13]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-13-56/task_gpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics_gpu(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-13-56/task_gpu_fi


## 1.1. MLP Results

In [14]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),98.553333,98.256667,98.656667,98.776667,98.933333,99.090000
Training time (s),42.100000,44.933333,63.433333,60.766667,60.900000,58.166667
RAM Usage (MB),2.412760,2.627604,2.160156,2.578125,2.524740,2.602865
CPU Usage (s),70.566667,73.463333,92.160000,90.320000,89.673333,89.010000
GPU Usage (%),6.333333,4.000000,12.214286,8.968254,12.928571,9.555556
GPU Memory (MB),88.000000,12.000000,60.000000,6.000000,60.000000,6.000000


## 1.2. CNN Results

In [15]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),98.823333,98.863333,99.160000,99.130000,99.426667,99.496667
Training time (s),48.100000,44.333333,62.666667,61.366667,64.200000,61.233333
RAM Usage (MB),4.385417,3.906250,2.516927,2.378906,2.441406,2.436198
CPU Usage (s),76.273333,72.630000,92.166667,90.550000,93.680000,90.340000
GPU Usage (%),8.833333,6.066667,16.000000,10.523810,15.238095,11.015873
GPU Memory (MB),88.666667,12.000000,40.000000,4.000000,40.000000,4.000000


# 2. Task-IL + No-GPU

In [16]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-15-21/task_nogpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-15-21/task_nogpu_fi


## 2.1. MLP Results

In [20]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)' , 
                                 'CPU Usage (s)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),98.623333,99.020000,98.880000,98.853333,99.116667,99.113333
Training time (s),43.700000,37.166667,279.166667,190.900000,283.833333,190.200000
RAM Usage (MB),115.717448,17.430990,77.079427,20.729167,90.412760,20.975260
CPU Usage (s),145.380000,103.570000,1666.160000,994.530000,1697.943333,992.533333


## 2.2. CNN Results

In [21]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),98.983333,98.760000,99.403333,99.206667,99.296667,99.333333
Training time (s),52.366667,47.100000,369.666667,233.100000,365.333333,233.533333
RAM Usage (MB),129.235677,26.395833,71.998698,28.005208,86.898438,27.498698
CPU Usage (s),191.670000,154.710000,2237.673333,1268.086667,2207.973333,1270.123333


# 3. Class-IL + GPU

In [22]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-18-09/class_gpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics_gpu(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-18-09/class_gpu_fi


## 3.1. MLP Results

In [23]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),94.703333,94.690000,93.950000,94.493333,93.193333,94.523333
Training time (s),93.100000,89.300000,106.600000,115.133333,111.233333,112.866667
RAM Usage (MB),2.157552,2.596354,1.970052,2.067708,2.255208,2.549479
CPU Usage (s),148.996667,145.186667,164.520000,174.053333,168.786667,171.253333
GPU Usage (%),5.700000,3.937037,14.921717,9.277778,13.820707,9.083333
GPU Memory (MB),137.333333,18.666667,60.000000,6.000000,60.000000,6.000000


## 3.2. CNN Results

In [25]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),96.746667,94.420000,94.856667,92.153333,94.640000,94.443333
Training time (s),184.066667,184.600000,250.533333,230.333333,246.800000,234.500000
RAM Usage (MB),3.270833,3.472656,2.075521,2.971354,2.417969,2.604167
CPU Usage (s),295.366667,295.696667,366.670000,343.803333,359.266667,346.363333
GPU Usage (%),8.694932,5.727193,16.067654,10.950145,16.243419,10.666667
GPU Memory (MB),138.000000,18.000000,40.000000,4.000000,40.000000,4.000000


# 4. Class-IL + No-GPU

In [30]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-21-25/class_nogpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-02-21-25/class_nogpu_fi


## 4.1. MLP Results

In [31]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)' , 
                                 'CPU Usage (s)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),94.850000,94.586667,93.826667,94.663333,93.290000,94.633333
Training time (s),86.400000,75.933333,567.700000,412.033333,566.666667,419.833333
RAM Usage (MB),204.865885,24.701823,84.944010,35.761719,100.621094,36.368490
CPU Usage (s),288.266667,214.773333,3404.523333,2195.426667,3399.503333,2234.736667


## 4.2. CNN Results

In [32]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)' , 
                                 'CPU Usage (s)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),96.680000,94.803333,95.196667,91.633333,94.973333,93.556667
Training time (s),221.200000,189.633333,1518.266667,993.133333,1494.366667,984.166667
RAM Usage (MB),210.085938,34.397135,109.505208,71.895833,114.552083,73.971354
CPU Usage (s),814.846667,631.273333,9215.200000,5478.453333,9063.303333,5399.463333
